## Batch Normalization and Dropout

Discover how batch normalization and dropout improve a model's accuracy.

We will be covering:

- Batch Normalization

- Notations

- Advantages and disadvantages of using batch normalization

- Dropout

### Batch Normalization

If you open any introductory machine learning textbook, you will find the idea of **input scaling**. It is undesirable to train a model with **gradient descent** with non-normalized input features.

Let’s start with an intuitive example to understand why we want normalization inside any model.

Suppose you have an input feature $x1$ in the range [0,10000] and another feature $x2$ in the range [0,1]. Any linear combination would ignore $x2$ such that $x1*w1 + x2*w2 \approx x1$, since our weights are initialized in a very tiny range like [-1,1].

We encounter the same issues inside the layers of deep neural networks. In this lesson, we will propagate this idea inside the NN.

> If we think out of the box, any intermediate layer is conceptually the same as the input layer; it accepts features and transforms them

### Notations

Throughout this lesson, $N$ will be the batch size, $H$ will refer to the height, $W$ to the width, and $C$ to the feature channels. The greek letter $\mu()$ refers to mean and the greek letter $\sigma()$ refers to standard deviation.

The batch features are denoted by $x$ with a shape of $[N, C, H, W]$.

![pic](https://raw.githubusercontent.com/CUTe-EmbeddedAI/images/main/images/fig22.PNG)

We will visualize the 4D activation maps x by **merging the spatial dimensions**. Now, we have a 3D shape that looks like this:

![pic](https://raw.githubusercontent.com/CUTe-EmbeddedAI/images/main/images/fig23.PNG)

The most dominant solution is batch normalization. Let’s see how it works.

> Batch Normalization (BN) normalizes the mean and standard deviation **for each individual feature channel/map**.

First of all, the mean and standard deviation are first-order statistics, and thus they relate to the **global characteristics** (such as the image style).

In this way, we somehow blend the global characteristics. Such a strategy is effective when we want our representation to share these characteristics. This is the reason that we widely utilize BN in downstream tasks (i.e., image classification).

From a mathematical point of view, **you can think of it as bringing the features of the image in the same range**.

![pic](https://raw.githubusercontent.com/CUTe-EmbeddedAI/images/main/images/fig24.PNG)

Specifically, we demand from our features to follow a Gaussian distribution with zero mean and unit variance. Mathematically, this can be expressed as:

![pic](https://raw.githubusercontent.com/CUTe-EmbeddedAI/images/main/images/fig25.PNG)

The index $c$ denotes the per-channel (feature map) mean.

Let’s see this operation visually:

![pic](https://raw.githubusercontent.com/CUTe-EmbeddedAI/images/main/images/fig26.PNG)

Notably, the spatial dimensions as well as the image batch are averaged. This way, **we concentrate our features in a compact Gaussian-like space**, which is usually beneficial.

In fact, $\gamma$ and $\beta$ correspond to the trainable parameters that result in the linear/affine transformation, which is different for all channels.

Specifically $\gamma$ and $\beta$ are vectors with the channel dimensionality.

### Advantages and disadvantages of using batch normalization

The following are some **advantages** of BN:

- BN accelerates the training of deep neural networks and tackles the vanishing gradient problem.

- For every input mini-batch, we calculate different statistics. This introduces some sort of regularization. Regularization refers to any form of technique/constraint that restricts the complexity of a deep neural network during training.

- BN also has a beneficial effect on the gradient flow through the network. It reduces the dependence of gradients on the scale of the parameters or of their initial values. This allows us to use much higher learning rates.

- In theory, BN makes it possible to use saturating nonlinearities by preventing the network from getting stuck, but we just use nn.ReLU().

- BN makes the gradients more predictive.

BN has the following **disadvantages**:

- Batch normalization may cause inaccurate estimation of batch statistics when we have a small batch size. This increases the model error. In tasks such as image segmentation, the batch size is usually too small. BN needs a sufficiently large batch size.

Let’s now implement batch normalization from scratch for images of size $[N, C, H, W]$. All you have to do is transform the above equations to Pytorch. The tricky part is to correctly figure out the sizes of each tensor.

In [1]:
import torch

# Gamma and beta are provided as 1d tensors. 
# X is the data in a mini-batch

def batchnorm(X, gamma, beta):

    # extract the dimensions
    N, C, H, W = list(X.size())
    # mini-batch mean
    mean = torch.mean(X, dim=(0, 2, 3))
    # mini-batch variance
    variance = torch.mean((X - mean.reshape((1, C, 1, 1))) ** 2, dim=(0, 2, 3))
    # normalize
    X_hat = (X - mean.reshape((1, C, 1, 1))) * 1.0 / torch.sqrt(variance.reshape((1, C, 1, 1)) )
    # scale and shift
    out = gamma.reshape((1, C, 1, 1)) * X_hat + beta.reshape((1, C, 1, 1))

    return out  

### Dropout

Another technique to train deep learning models is dropout.

Conceptually, dropout approximates training a large number of neural networks with different architectures in parallel.

> In practice, during training, some number of layer outputs are randomly ignored (dropped out) with probability $p$. 

Thus, the same layer will alter its connectivity and will search for alternative paths to convey the information in the next layer. As a result, each update to a layer during training is performed with a different “view” of the configured layer.

“Dropping” values means temporarily removing them from the network for the current forward pass along with all its incoming and outgoing connections.

Dropout has the effect of making the training process noisy. The choice of the probability $p$ depends on the architecture.

![pic](https://raw.githubusercontent.com/CUTe-EmbeddedAI/images/main/images/fig27.PNG)
This conceptualization suggests that perhaps dropout breaks-up situations where network layers co-adapt to correct mistakes from prior layers, in turn making the model more robust.

The neural network will adapt in a way that prevents overfitting, which refers to poor generalization to unseen data.

Dropout increases the sparsity of the network and in general encourages sparse representations!

You can find an example in the code below .

Notice that each value will be zeroed with a probability of p=0.5. Nonetheless, that doesn’t imply that the output will be 50% zeroed-out every time.

In [2]:
import torch
import torch.nn as nn

inp = torch.rand(1,8)
layer = nn.Dropout(0.5)
out1 = layer(inp)
out2 = layer(inp)
print(inp)
print(out1)
print(out2)

tensor([[0.6593, 0.3519, 0.6087, 0.8115, 0.4393, 0.6048, 0.5635, 0.0887]])
tensor([[0.0000, 0.0000, 1.2174, 1.6230, 0.8786, 0.0000, 1.1270, 0.0000]])
tensor([[1.3187, 0.7039, 1.2174, 0.0000, 0.0000, 1.2096, 1.1270, 0.0000]])


### Training with batchnorm

Now, it's the time to apply batchnorm and dropout! Let's copy and paste the code for training a CNN in our previous notebook. 

Let's make the necessary imports and data preparation as we did before.

In [3]:
import os

#Numpy is linear algebra lbrary
import numpy as np
# Matplotlib is a visualizations library 
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [4]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4


trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
       'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

train_data_size = len(trainloader.dataset)
test_data_size = len(testloader.dataset)

print(train_data_size)
print(test_data_size)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
50000
10000


Here, try to incorporate batchnorm as a layer in this vanilla CNN.

Note that in pytorch, batchnorm can be implemented using `nn.BatchNorm2d(num_features)` for 2D input and `nn.BatchNorm2d(num_features)` for 1D input. 

In [5]:
#1. DEFINE THE CNN WITH BATCHNORM
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
#         self.batchnorm = nn.BatchNorm2d(6),
#         self.batchnorm = batchnorm()
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.relu = nn.ReLU()
        # DEFINE BATCHNORM LAYER HERE #

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        # INCLUDE BATCHNORM IN THE FORWARD METHOD #
        x = x.view(-1, 16 * 5 * 5)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [6]:
model = CNN() # need to instantiate the network to be used in instance method

# 2. LOSS AND OPTIMIZER
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 3. move the model to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

CNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (relu): ReLU()
)

In [7]:
import time # to calculate training time

def train_and_validate(model, loss_criterion, optimizer, epochs=25):
    '''
    Function to train and validate
    Parameters
        :param model: Model to train and validate
        :param loss_criterion: Loss Criterion to minimize
        :param optimizer: Optimizer for computing gradients
        :param epochs: Number of epochs (default=25)
  
    Returns
        model: Trained Model with best validation accuracy
        history: (dict object): Having training loss, accuracy and validation loss, accuracy
    '''
    
    start = time.time()
    history = []
    best_acc = 0.0

    for epoch in range(epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch+1, epochs))
        
        # Set to training mode
        model.train()
        
        # Loss and Accuracy within the epoch
        train_loss = 0.0
        train_acc = 0.0
        
        valid_loss = 0.0
        valid_acc = 0.0
        
        for i, (inputs, labels) in enumerate(trainloader):

            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Clean existing gradients
            optimizer.zero_grad()
            
            # Forward pass - compute outputs on input data using the model
            outputs = model(inputs)
            
            # Compute loss
            loss = loss_criterion(outputs, labels)
            
            # Backpropagate the gradients
            loss.backward()
            
            # Update the parameters
            optimizer.step()
            
            # Compute the total loss for the batch and add it to train_loss
            train_loss += loss.item() * inputs.size(0)
            
            # Compute the accuracy
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))
            
            # Convert correct_counts to float and then compute the mean
            acc = torch.mean(correct_counts.type(torch.FloatTensor))
            
            # Compute total accuracy in the whole batch and add to train_acc
            train_acc += acc.item() * inputs.size(0)
            
            #print("Batch number: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}".format(i, loss.item(), acc.item()))

            
        # Validation - No gradient tracking needed
        with torch.no_grad():

            # Set to evaluation mode
            model.eval()

            # Validation loop
            for j, (inputs, labels) in enumerate(testloader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Forward pass - compute outputs on input data using the model
                outputs = model(inputs)

                # Compute loss
                loss = loss_criterion(outputs, labels)

                # Compute the total loss for the batch and add it to valid_loss
                valid_loss += loss.item() * inputs.size(0)

                # Calculate validation accuracy
                ret, predictions = torch.max(outputs.data, 1)
                correct_counts = predictions.eq(labels.data.view_as(predictions))

                # Convert correct_counts to float and then compute the mean
                acc = torch.mean(correct_counts.type(torch.FloatTensor))

                # Compute total accuracy in the whole batch and add to valid_acc
                valid_acc += acc.item() * inputs.size(0)

                #print("Validation Batch number: {:03d}, Validation: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))
            
        # Find average training loss and training accuracy
        avg_train_loss = train_loss/train_data_size 
        avg_train_acc = train_acc/train_data_size

        # Find average training loss and training accuracy
        avg_test_loss = valid_loss/test_data_size 
        avg_test_acc = valid_acc/test_data_size

        history.append([avg_train_loss, avg_test_loss, avg_train_acc, avg_test_acc])
                
        epoch_end = time.time()
    
        print("Epoch : {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\tValidation : Loss : {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(epoch, avg_train_loss, avg_train_acc*100, avg_test_loss, avg_test_acc*100, epoch_end-epoch_start))
        
        # Save if the model has best accuracy till now
        torch.save(model, 'cifar10_model_'+str(epoch)+'.pt')
            
    return model, history

In [ ]:
# 4. Train the model for 10 epochs

num_epochs = 10
trained_model, history = train_and_validate(model, criterion, optimizer, num_epochs)

Epoch: 1/10
Epoch : 000, Training: Loss: 1.6285, Accuracy: 40.0720%, 
		Validation : Loss : 1.4729, Accuracy: 45.7100%, Time: 78.6272s
Epoch: 2/10


### Training with dropout

Adding dropout to your PyTorch models is very straightforward with the `torch.nn.Dropout` class, which takes in the dropout rate – the probability of a neuron being deactivated – as a parameter.

Go back to the CNN class and try to incorporate dropout as an additional layer in the fully connected layer.